<a href="https://colab.research.google.com/github/ben854719/Threat-Detection-System/blob/main/Threat_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install libboost-all-dev

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.2 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,783 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,209 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,297 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:14 http://archive.ubuntu.co

In [ ]:
%%writefile AlarmSystem.cpp

#include <iostream>
#include <fstream>
#include <string>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include "RansomWare.detector"

int main() {
    const char* pipe_path = "/tmp/ThreatPipe";

    // Create the named pipe (FIFO) if it doesn't exist
    mkfifo(pipe_path, 0666);

    // Open the named pipe for writing
    int pipe_fd = open(pipe_path, O_WRONLY);

    if (pipe_fd == -1) {
        std::cerr << "Failed to open pipe." << std::endl;
        return 1;
    }

    std::string message = "entropy=7.8,file_size=204800,prediction=1";
    write(pipe_fd, message.c_str(), message.size());

    close(pipe_fd);
    return 0;
}

Overwriting AlarmSystem.cpp


In [10]:
!g++ AlarmSystem.cpp -o AlarmSystem
!./AlarmSystem

^C


In [7]:
%%writefile RansomWare.detector.hpp

#include <windows.h>
#include <iostream>
#include <string>

void MonitorDirectory(const std::wstring& path) {
    HANDLE hDir = CreateFileW(
        path.c_str(), FILE_LIST_DIRECTORY,
        FILE_SHARE_READ | FILE_SHARE_WRITE | FILE_SHARE_DELETE,
        NULL, OPEN_EXISTING, FILE_FLAG_BACKUP_SEMANTICS, NULL);

    if (hDir == INVALID_HANDLE_VALUE) {
        std::wcerr << L"Failed to open directory." << std::endl;
        return;
    }

    char buffer[1024];
    DWORD bytesReturned;

    while (true) {
        if (ReadDirectoryChangesW(hDir, buffer, sizeof(buffer), TRUE,
            FILE_NOTIFY_CHANGE_FILE_NAME | FILE_NOTIFY_CHANGE_LAST_WRITE,
            &bytesReturned, NULL, NULL)) {
            std::cout << "Suspicious file activity detected!" << std::endl;
            // Send to Python or log to file
        }
    }
}

Writing RansomWare.detector.hpp


In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib
import os

# In a real scenario, you would train and save your actual model
dummy_model = RandomForestClassifier()

# Create some dummy data to fit the model
dummy_data = pd.DataFrame({'entropy': [1, 2, 3], 'file_size': [100, 200, 300]})
dummy_labels = [0, 0, 1] # 0 for not ransomware, 1 for ransomware
dummy_model.fit(dummy_data, dummy_labels)

# Save the dummy model
model_filename = "ransomware_model.pkl"
joblib.dump(dummy_model, model_filename)

print(f"Dummy model saved as {model_filename}")

Dummy model saved as ransomware_model.pkl


In [14]:
import pandas as pd
import joblib

# Load the dummy model
model_filename = "ransomware_model.pkl"
model = joblib.load(model_filename)

def classify_event(event_data):
    """
    Classifies an event as ransomware or not.

    Args:
        event_data (dict): A dictionary containing event features like 'entropy' and 'file_size'.

    Returns:
        int: 1 if classified as ransomware, 0 otherwise.
    """
    # Convert event data to a pandas DataFrame
    event_df = pd.DataFrame([event_data])

    # Make a prediction using the loaded model
    prediction = model.predict(event_df)

    return int(prediction[0])

# Test the classify_event function with sample data
sample_event_1 = {'entropy': 2.5, 'file_size': 150}
prediction_1 = classify_event(sample_event_1)
print(f"Sample Event 1 ({sample_event_1}): Prediction = {prediction_1}")

sample_event_2 = {'entropy': 7.8, 'file_size': 204800}
prediction_2 = classify_event(sample_event_2)
print(f"Sample Event 2 ({sample_event_2}): Prediction = {prediction_2}")

Sample Event 1 ({'entropy': 2.5, 'file_size': 150}): Prediction = 0
Sample Event 2 ({'entropy': 7.8, 'file_size': 204800}): Prediction = 1
